# Bonus

In [35]:
import pandas as pd
import numpy as np

from sklearn.model_selection import cross_validate
from category_encoders import TargetEncoder
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge, Lasso, LinearRegression
from sklearn.metrics import make_scorer, mean_squared_error, r2_score
from sklearn.ensemble import GradientBoostingRegressor

In [2]:
transactions_df = pd.read_csv("https://www.dropbox.com/s/360xhh2d9lnaek3/allegro-api-transactions.csv?dl=1")

In [3]:
transactions_df = transactions_df[["main_category", "categories", "it_location", "price"]]
transactions_df.head()

,main_category,categories,it_location,price
0,Komputery,"['Komputery', 'Dyski i napędy', 'Nośniki', 'No...",Warszawa,59.99
1,"Odzież, Obuwie, Dodatki","['Odzież, Obuwie, Dodatki', 'Bielizna damska',...",Warszawa,4.90
2,Dom i Ogród,"['Dom i Ogród', 'Budownictwo i Akcesoria', 'Śc...",Leszno,109.90
3,Książki i Komiksy,"['Książki i Komiksy', 'Poradniki i albumy', 'Z...",Wola Krzysztoporska,18.50
4,"Odzież, Obuwie, Dodatki","['Odzież, Obuwie, Dodatki', 'Ślub i wesele', '...",BIAŁYSTOK,19.90


In [4]:
transactions_df["it_location"] = pd.DataFrame(transactions_df["it_location"].str.lower())
transactions_df["it_location"] = pd.DataFrame(transactions_df["categories"].str.lower())
transactions_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 420020 entries, 0 to 420019
Data columns (total 4 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   main_category  420020 non-null  object 
 1   categories     420020 non-null  object 
 2   it_location    420020 non-null  object 
 3   price          420020 non-null  float64
dtypes: float64(1), object(3)
memory usage: 12.8+ MB


## Modeling
### Target encoding
Poniższe informacje wzięte są z [https://maxhalford.github.io/blog/target-encoding/](https://maxhalford.github.io/blog/target-encoding/) .


Użyjemy tutaj target encodingu z parametrem `smoothing`, dzięki czemu do kodowania zamiast zwykłej średniej targetu po grupach wykorzystana zostanie wartość $$\frac{n \times \bar{x} + m \times w}{n + m},$$
gdzie
- $n$ to liczność danej grupy,
- $\bar{x}$ to średnia targetu z danej grupy,
- $w$ to ogólna średnia targetu,
- $m$ to wartość `smoothing`($m \in [0, \infty)$).

Taki encoding jest użyteczny, np. gdy wiele grup jest nielicznych. Wtedy sama średnia może z dużym błędem estymować wartość oczekiwaną targetu w tych grupach, więc pomocne może być uwzględnienie średniej ogólnej. Wartość `smoothing` określa jak bardzo polegamy na ogólnej średniej targetu.

### Regularyzacja
Polega na wprowadzeniu dodatkowego bias'u(błędu na danych treningowych), aby zmniejszyć variance(błąd na danych testowych). Jedne z najbardziej znanych technik to 
- `l1`(Lasso), kara w postaci kwadratu 2 normy wektora wag(z regresji liniowej),  
- `l2`(Ridge), kara w postaci 1 normy wektora wag.

Dane techniki użyteczne są np. przy obecności skorelowanych zmiennych lub mało istotnych cech.

In [39]:
X, y = transactions_df.drop("price", axis=1), transactions_df["price"]

def get_scores(model, X, y):
    scoring = {
    "r2": make_scorer(r2_score),
    "rmse": make_scorer(mean_squared_error, squared=False)
    }
    scores = cross_validate(model, X, y, 
                         scoring=scoring,
                         cv=5,
                         n_jobs=-1)
    return scores


def get_model(model_type):

    if model_type == "lasso":
        clf = Lasso()
    elif model_type == "ridge":
        clf = Ridge()
    elif model_type == "least_sq":
        clf = LinearRegression()
    elif model_type == "gbc":
        clf =  GradientBoostingRegressor()
    else:
        raise NameError("model type: " + model_type + " is unknown")

    pipeline =  Pipeline([("encoder", TargetEncoder(smoothing=300)), ("clf", clf)])

    return pipeline


def experiment(X, y):
    model_types = ["least_sq", "lasso", "ridge", "gbc"]
    
    res_merged = None
    for i in range(len(model_types)):
        model = get_model(model_types[i])
        scores = get_scores(model, X, y)
        res = pd.DataFrame.from_dict({"model": [model_types[i]],
                                     "r2_mean": [np.mean(scores["test_r2"])],
                                     "r2_std": [np.std(scores["test_r2"])],
                                     "rmse_mean": [np.mean(scores["test_rmse"])],
                                     "rmse_std": [np.std(scores["test_rmse"])]})
        res_merged = pd.concat([res_merged, res], axis=0).reset_index(drop=True)
        
    return res_merged

## Wnioski
Wszystkie modele miały słabe wyniki. Najlepiej się sprawdził gradient boosting regressor(miał najwyższy `r2_score` i najniższe `rmse`). Miał on także najbardziej stabilne wyniki(najmniejsze odchylenie). Jeśli chodzi o regularyzację, to w danym zadaniu ona niestety nie pomogła.

In [40]:
experiment(X, y)

,model,r2_mean,r2_std,rmse_mean,rmse_std
0,least_sq,0.129755,0.062184,359.396395,82.477539
1,lasso,0.129759,0.062185,359.395656,82.477432
2,ridge,0.129755,0.062184,359.396395,82.477539
3,gbc,0.168278,0.058741,351.094369,80.166762
